Our first method will use simple counting i.e. count the number of times each character is predicted based on two characters provided e.g. the name jane would defines n as the next characters after 'ja' so that would add to 'n' probability of being the next character 

In [546]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
%pip install torch torchvision
import torch

Note: you may need to restart the kernel to use updated packages.


In [547]:
# Read in words
words = open('names.txt', 'r').read().splitlines()
words[:10]


['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In this instance, we are only looking at three characters. Two input characters, and the character we are predicting next.

e.g. what characters are likely to follow r

we also now what names are likely to start and finish at


We then do a simple count of most prominent trigrams

How many examples do we get from emma?

<S>e -> m
em -> m
mm -> a
ma -> <E>

In [569]:
t = {}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] # hallucinate a start character and end character 
    for pair, ch2 in zip(zip(chs, chs[1:]), chs[2:]):
        trigram  = (''.join(pair), ch2)
        t[trigram] = t.get(trigram, 0) + 1

{('<S>e', 'm'): 288,
 ('em', 'm'): 100,
 ('mm', 'a'): 72,
 ('ma', '<E>'): 174,
 ('<S>o', 'l'): 104,
 ('ol', 'i'): 69,
 ('li', 'v'): 54,
 ('iv', 'i'): 78,
 ('vi', 'a'): 147,
 ('ia', '<E>'): 903,
 ('<S>a', 'v'): 243,
 ('av', 'a'): 161,
 ('va', '<E>'): 93,
 ('<S>i', 's'): 124,
 ('is', 'a'): 142,
 ('sa', 'b'): 76,
 ('ab', 'e'): 173,
 ('be', 'l'): 201,
 ('el', 'l'): 822,
 ('ll', 'a'): 337,
 ('la', '<E>'): 684,
 ('<S>s', 'o'): 152,
 ('so', 'p'): 21,
 ('op', 'h'): 37,
 ('ph', 'i'): 61,
 ('hi', 'a'): 81,
 ('<S>c', 'h'): 352,
 ('ch', 'a'): 236,
 ('ha', 'r'): 329,
 ('ar', 'l'): 287,
 ('rl', 'o'): 44,
 ('lo', 't'): 14,
 ('ot', 't'): 34,
 ('tt', 'e'): 121,
 ('te', '<E>'): 175,
 ('<S>m', 'i'): 393,
 ('mi', 'a'): 95,
 ('<S>a', 'm'): 384,
 ('am', 'e'): 226,
 ('me', 'l'): 188,
 ('el', 'i'): 537,
 ('li', 'a'): 518,
 ('<S>h', 'a'): 505,
 ('ar', 'p'): 8,
 ('rp', 'e'): 5,
 ('pe', 'r'): 77,
 ('er', '<E>'): 683,
 ('<S>e', 'v'): 154,
 ('ev', 'e'): 142,
 ('ve', 'l'): 76,
 ('el', 'y'): 353,
 ('ly', 'n'): 976,


Now lets get the count of each combo and sort. we'll see that ah followed by ending character occurs the

In [549]:
sorted_t = sorted(t.items(), key = lambda kv: -kv[1])
sorted_t[:10]

[(('ah', '<E>'), 1714),
 (('na', '<E>'), 1673),
 (('an', '<E>'), 1509),
 (('on', '<E>'), 1503),
 (('<S>m', 'a'), 1453),
 (('<S>j', 'a'), 1255),
 (('<S>k', 'a'), 1254),
 (('en', '<E>'), 1217),
 (('ly', 'n'), 976),
 (('yn', '<E>'), 953)]

Lets convert our trigram to an array. We need to deduce how many rows we have, so it should be all the combinations of characters against our list of 28 chars.

28 characters consist of 26 alphabet characters plus our <S> and <E> characters. So as we can pair same characters together e.g. -> a,a and with the other 27 chars, there is a total of 28*28 combinations -> 784

In [563]:
import string

# Create a list of all lowercase letters
letters = string.ascii_lowercase

# Create a list of tuples where the first element is a pair of characters and the second element is an integer index
pairs_string_to_integer = [(a+b, i) for i, (a, b) in enumerate((x, y) for x in letters for y in letters)]

# Print the first 10 items
print(pairs_string_to_integer[:10])

[('aa', 0), ('ab', 1), ('ac', 2), ('ad', 3), ('ae', 4), ('af', 5), ('ag', 6), ('ah', 7), ('ai', 8), ('aj', 9)]


In [564]:
## Creates a 784 * 28 tensor
import torch

N = torch.zeros((len(pairs_string_to_integer), 28), dtype=torch.int32) # 28*28 = 784 for different pair combos
N[:5]

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]], dtype=torch.int32)

We need to create a lookup of integers to pairs so we can have an index. 
Basically we need an integer ref for all our points in our torch, that means each character pair needs an integer value, and each character needs a value e.g. ac = [100] and if it predicts d as next letter it would be axis point [100, 3]. Therefore each time this is shown, we add +1, allowing us to count the number of occurences 

If I understand this correctly, we therefore need two integer lookups: 
-
Taking our char pairs as the 'y' axis - we need 0:len(unique-pairs)  i.e. 0-600 and then our single character axis being x axis, we need 28 chars (alphabet including <S> and <E>)

0   |------------------------------28
    |<S>aa   <S>ab  <S>ac
    |aaa     aab    aac
    |..
    |..
    |..
784 |zaa zab

Lets get the character string to integer index for our 'y-axis' which is a - z and our <S> and <E> characters

In [578]:
# returns all the set of lowercase characters
chars = sorted(set(''.join(words)))
char_to_integer = {s:i for i,s in enumerate(chars)}
char_to_integer['<S>'] = 26
char_to_integer['<E>'] = 27
print(list(char_to_integer)[:10])

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']


Now add our counts of each character to the tensor i.e. all counts of aab -> zzz occuring


In [592]:
print(pairs_string_to_integer['<S>e'])

TypeError: list indices must be integers or slices, not str

In [594]:
for w in words:
    chs = ['<S>'] + list(w) + ['<E>'] # hallucinate a start character and end character 
    for pair, ch2 in zip(zip(chs, chs[1:]), chs[2:]):
        trigram  = (''.join(pair), ch2) #  e.g. ('<S>e', 'm')
        pairs_combined = trigram[0] #  e.g '<S>e'
        ix1 = pairs_string_to_integer[pairs_combined]
        print(ix1)
        
        # ix2 = char_to_integer[ch2]
        # N[ix1, ix2] += 1  

<S>e
em
mm
ma
<S>o
ol
li
iv
vi
ia
<S>a
av
va
<S>i
is
sa
ab
be
el
ll
la
<S>s
so
op
ph
hi
ia
<S>c
ch
ha
ar
rl
lo
ot
tt
te
<S>m
mi
ia
<S>a
am
me
el
li
ia
<S>h
ha
ar
rp
pe
er
<S>e
ev
ve
el
ly
yn
<S>a
ab
bi
ig
ga
ai
il
<S>e
em
mi
il
ly
<S>e
el
li
iz
za
ab
be
et
th
<S>m
mi
il
la
<S>e
el
ll
la
<S>a
av
ve
er
ry
<S>s
so
of
fi
ia
<S>c
ca
am
mi
il
la
<S>a
ar
ri
ia
<S>s
sc
ca
ar
rl
le
et
tt
<S>v
vi
ic
ct
to
or
ri
ia
<S>m
ma
ad
di
is
so
on
<S>l
lu
un
na
<S>g
gr
ra
ac
ce
<S>c
ch
hl
lo
oe
<S>p
pe
en
ne
el
lo
op
pe
<S>l
la
ay
yl
la
<S>r
ri
il
le
ey
<S>z
zo
oe
ey
<S>n
no
or
ra
<S>l
li
il
ly
<S>e
el
le
ea
an
no
or
<S>h
ha
an
nn
na
ah
<S>l
li
il
ll
li
ia
an
<S>a
ad
dd
di
is
so
on
<S>a
au
ub
br
re
ey
<S>e
el
ll
li
ie
<S>s
st
te
el
ll
la
<S>n
na
at
ta
al
li
ie
<S>z
zo
oe
<S>l
le
ea
ah
<S>h
ha
az
ze
el
<S>v
vi
io
ol
le
et
<S>a
au
ur
ro
or
ra
<S>s
sa
av
va
an
nn
na
ah
<S>a
au
ud
dr
re
ey
<S>b
br
ro
oo
ok
kl
ly
yn
<S>b
be
el
ll
la
<S>c
cl
la
ai
ir
re
<S>s
sk
ky
yl
la
ar
<S>l
lu
uc
cy
<S>p
pa
ai
is
sl
le
ey
<S

In [ ]:
N[10]

In [ ]:
%pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Create an inverse of string to integer

In [ ]:
pred_itos = {i:s for s,i in pred_string_to_integer.items()}
pair_itos = {i:s for s,i in pairs_string_to_integer.items()}

In [ ]:
# To do: FIX THIS
plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(len(pair_itos)):
    for j in range(len(pred_itos)):
        chstr = pair_itos[i] + pred_itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color="gray")
plt.axis('off')

In [ ]:
N[10]

Now lets convert to probabilitiess. Gives us the probability of any character being the first character of a name. 

In [ ]:
P = (N+0.01).float()  
P /=  P.sum(1, keepdims=True)
P[1]

For sampling we will use torch.multinomial and a generator object to make everything deterministic 

This line of code is using PyTorch, a popular machine learning library in Python. 

The `torch.Generator()` is an object that holds the state of the random number generator. You can think of it as a container for the algorithm that produces pseudo-random numbers.

The `manual_seed()` function is used to set the seed for generating random numbers. This ensures that the random numbers generated are deterministic, meaning if you use the same seed, you will get the same sequence of random numbers. This is useful for debugging and testing purposes, as it allows for reproducibility in your code.

In [ ]:
g = torch.Generator().manual_seed(214743647)

Logic -> We predict the 3rd character from the beginning two e.g. em -> a. 
We then strip the first character and append our 3rd character for the next prediction i.e. ma -> b

- Starting off the prediction, we need to start on <S>(x) character

In [ ]:
start_chars = []
for key, value in pair_itos.items():
    if '<S>' in str(value):
        start_chars.append((key, value))
start_chars[:10]

We also need to know when to end the sequence, so we must find all characters with an ending character. This will be the value with pred = <E>

In [ ]:
import random
ix = random.choice(start_chars)


In [ ]:
iy_pred_reference = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
iy_pred_reference

In [ ]:
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
ix

Note - When we run through predictions, we need to predict like ab -> c -> bc -> d -> cd -> e
Therefore we need to combine the second character in the pair with the predicted value. This is demonstrated below.

Note: -1 gets the last character from a string

In [ ]:
ix_pair = pair_itos[10][-1] + pred_itos[10] 
ix_pair

### LATEST TO DO: We need to map every single type of pair from aa -> zz as we need to account for pairs that aren't predicted in our dataset
### e.g. bk is not predicted here, so it returns an error as the pair does not exist
### We could do an if not exists assign random character, but would be nicer to handle this inside the tensor
pairs_string_to_integer[ix_pair]

In [ ]:
import random
g = torch.Generator().manual_seed(214748347)

for i in range(10):
    # Select a random value from start_chars
    out = []
    start_ix = random.choice(start_chars) # Gets all pairs that start with <S> 
    ix_pair = start_ix[0] # Gets integer reference for pair e.g. fe -> 45
    while True:
        p = P[ix_pair]
        iy_pred_reference = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(pred_itos[iy_pred_reference])
        if '<E>' in pred_itos[iy_pred_reference]:
            break
        # Combines last letter of pair and predicted char e.g. xy predicts z  so ix_pair now equals  = yz 
        ix_pair = pair_itos[ix_pair][-1] + pred_itos[iy_pred_reference] 
    # print(''.join(out))


In [ ]:
print(t)
print(pairs)
# pred_string_to_integer = {s:i for i,s in enumerate(chars)}
# pred_string_to_integer['<S>'] = 26
# pred_string_to_integer['<E>'] = 27
# pred_string_to_integer

In [ ]:
print(pred_itos)